In [ ]:
#test oceanwind code with degenerations
from tensorflow.keras import backend as K
from tensorflow import keras

import matplotlib.pylab as plt
%matplotlib inline
from matplotlib import pyplot

import threading
import argparse
import pickle
import numpy as np
import pandas as pd
import csv
from sklearn.metrics import mean_absolute_error, mean_squared_error
import sklearn.metrics as metrics
import math
import time
import gc
from datetime import datetime
from collections import Iterable
from tensorflow.keras import losses
from statistics import mean
from tensorflow.keras.utils import plot_model
import tensorflow.python.keras.backend as K

from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import TensorBoard,ModelCheckpoint, EarlyStopping,Callback
from tensorflow.keras.regularizers import L1L2
from tensorflow.keras.optimizers import *
K.set_image_data_format('channels_first')

import seaborn as sns
sns.set()

import os


import tensorflow.compat.v1 as tf
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
gpu_options = tf.GPUOptions(allow_growth=True)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

start_time = time.time()


n_classes = 18
time_step1 = 3
#optim = Adam(lr = 0.0001)
#optim = Adam(lr = 0.00001, decay = 0.00001)
optim = Adam(learning_rate = 1e-4)
#sample_batch = 16
sample_batch = 10
number_stations = 18
NB_epoch = 650

def eval_metrics_on(predictions,labels):
    '''assuming this is a regression task; labels are continuous-valued floats
    
    returns most regression-related scores for the given predictions/targets as a dictionary:
    
        r2, mean_abs_error, mse, rmse, median_absolute_error, explained_variance_score
    '''
    #if len(labels[0])==2: #labels is list of data/labels pairs
        #labels = np.concatenate([l[1] for l in labels])
    #predictions = predictions[:,0]
    prdictions = np.array(predictions)
    labels = np.array(labels)
    
    r2                       = metrics.r2_score(labels, predictions)
    mean_abs_error           = np.abs(predictions - labels).mean()
    mse                      = ((predictions - labels)**2).mean()
    rmse                     = np.sqrt(mse)
    #median_absolute_error    = metrics.median_absolute_error(labels, predictions) # robust to outliers
    explained_variance_score = metrics.explained_variance_score(labels, predictions) # best score = 1, lower is worse
    return {'r2':r2, 'mean_abs_error':mean_abs_error, 'mse':mse, 'rmse':rmse, 
    'explained_variance_score':explained_variance_score}


def predict_graph(true_val,predict_val):
    for sn in range(number_stations):
        plt.figure(figsize=(6,3))
        plt.figure(sn+1)
        plt.title('_next_12hour station = ' + str(sn+1))
        plt.plot(true_val[:,sn::number_stations], c='g', label= 'gt')
        plt.plot(predict_val[:,sn::number_stations], c='r', label= 'pred')
        plt.legend(loc='best',prop={'size': 5})
        plt.legend(loc='best')
        name = str("firstgroup_stat_") + str(sn) + str(".png")
        #plt.savefig(name)
        plt.show()
        
class ClearMemory(Callback):
    def on_epoch_end(self, epoch, logs=None):
        gc.collect()
        K.clear_session()
            
#extracted remote features
data_shape1 = (3,11720)
in1 = Input(shape=data_shape1)
#model1 = fn_model_schw(in1)

# local Pm2.5
data_shape2 = (1,18)
in2 = Input(shape=data_shape2)

#local weather
localweather_data_shape3 = (1,90)
in3 = Input(shape=localweather_data_shape3)


weather_in = Flatten()(in3)
weather_in = RepeatVector(time_step1)(weather_in)

pm_in = Flatten()(in2)
pm_in = RepeatVector(time_step1)(pm_in)

model_final_concat = concatenate([in1,pm_in,weather_in])


model_final_concat = TimeDistributed(Dense(52, activation='relu'))(model_final_concat)
#model_final_concat = Dense(128, activation='relu',bias_regularizer=L1L2(l1 = 0.01,l2=0.01))(model_final_concat)
model_final_concat = TimeDistributed(Dropout(rate = 0.5))(model_final_concat)
#model_final_concat = BatchNormalization()(model_final_concat)
#model_final_concat = Dense(64, activation='relu',bias_regularizer=L1L2(l1 = 0.01,l2=0.01))(model_final_concat)
model_final_concat = TimeDistributed(Dense(42, activation='relu'))(model_final_concat)
model_final_concat = TimeDistributed(Dropout(rate=0.5))(model_final_concat)
#model_final_concat = Dropout(rate=0.3)(model_final_concat)

model_final_concat = TimeDistributed(Dense(n_classes))(model_final_concat)


model = Model(inputs=[in1,in2,in3],outputs=model_final_concat)

model.summary()
                      

aod_train = np.load('middlelayer_train_3day_4tile.npy').astype('float')
aod_valid = np.load('middlelayer16_3day_4tile.npy').astype('float')

local_weather_train14 = np.load('local_weather_yr14_n3day_train.npy').astype('float')
local_weather_train15 = np.load('local_weather_yr15_n3day_train.npy').astype('float')
local_weather_train = np.concatenate((local_weather_train14,local_weather_train15),axis = 0)

local_weather_train = local_weather_train.reshape(local_weather_train.shape[0],local_weather_train.shape[1],
                                                  local_weather_train.shape[2]*local_weather_train.shape[3])

local_pm = np.load('pm_n3day_daily_train.npy')[:-6].astype('float')
tru_val = np.load('pm_label_n3day_daily_train.npy')[:-6].astype('float')

local_weather_test = np.load('local_weather_n3day_test.npy').astype('float')
local_weather_test = local_weather_test.reshape(local_weather_test.shape[0],local_weather_test.shape[1],
                                                local_weather_test.shape[2]*local_weather_test.shape[3])

local_pm_test = np.load('pm_n3day_daily_valid.npy')[:-4].astype('float')
tru_val_test = np.load('pm_label_n3day_valid_daily.npy')[:-4].astype('float')


model.compile(optimizer = optim, loss = 'mse', metrics = ['mse'], run_eagerly=True)

keras_callback = [
                EarlyStopping(monitor='val_mse', patience = 90, mode='min'),
                ModelCheckpoint(filepath='next3day_strip_model.h5', monitor='val_mse', 
                save_best_only=True,mode='min',verbose=1)
                ]


history = model.fit([aod_train,local_pm,local_weather_train[:-6]],
                            tru_val,batch_size = sample_batch,
                            epochs = NB_epoch,verbose = 1,shuffle = False,callbacks=keras_callback,
            validation_data = ([aod_valid,local_pm_test,local_weather_test[:-4]],
                                tru_val_test))


#summarize histoy for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','test'],loc = 'upper left')
plt.show()

print('this is val_mse')
plt.plot(history.history['mse'])
plt.plot(history.history['val_mse'])
plt.title('model mse')
plt.ylabel('mse')
plt.xlabel('epoch')
plt.legend(['train','test'],loc = 'upper left')
plt.show()                      

predictions_edit = model.predict([aod_valid,local_pm_test,local_weather_test[:-4]],
                            batch_size = sample_batch, verbose=1)

predictions = predictions_edit.reshape(predictions_edit.shape[0]*predictions_edit.shape[1],predictions_edit.shape[2])
print(predictions_edit.shape)
y_label = np.load('pm_label_n3day_valid_daily.npy')[:-4].astype("float")
y_label2 = y_label.reshape(y_label.shape[0]*y_label.shape[1],y_label.shape[2])
print('shape of true and predict ',y_label2.shape,predictions.shape)

#perform model evaluation
eval_results = eval_metrics_on(predictions,y_label2)
print('this is evaluation results for r2,mean_abs_error,mse,rmse,expl_var_score ',eval_results)
#print('this is evaluation results for r2,mean_abs_error,mse,rmse,expl_var_score ',eval_results)

#observe the graph between prediction and true value of PM2.5 for last hour
y_true = y_label[:,2:3,:]
y_true = y_true.reshape(y_true.shape[0]*y_true.shape[1],y_true.shape[2])
pred_graph = predictions_edit[:,0:1,:]
pred_graph =pred_graph.reshape(pred_graph.shape[0]*pred_graph.shape[1],pred_graph.shape[2])
predict_graph(y_true,pred_graph)

print('rmse value')
stat_dict = {}
stat_avg = []
for i in range(time_step1):
    t1 = predictions_edit[:,i:i+1,:]
    t1 = t1.reshape(t1.shape[0]*t1.shape[1],t1.shape[2])
    t2 = y_label[:,i:i+1,:]
    t2 = t2.reshape(t2.shape[0]*t2.shape[1],t2.shape[2])
    rsl = []
    for sn in range(number_stations):
        name = 'stats_' + str(sn)
        score2 = np.sqrt(np.mean(np.square(t1[:,sn::number_stations]-t2[:,sn::number_stations])))
        stat_dict.setdefault(name,[]).append(score2)
        #print(score2)
        rsl.append(score2)
for k in  stat_dict.keys():
    avg_value = sum(stat_dict[k])/time_step1
    #print(avg_value)
    stat_avg.append(avg_value)
print('mean of all stations ',mean(stat_avg))

In [ ]:
model2 = load_model('next3day_strip_model.h5')
predictions_edit2 = model2.predict([aod_valid,local_pm_test,local_weather_test[:-4]],
                            batch_size = sample_batch, verbose=1)

predictions = predictions_edit2.reshape(predictions_edit2.shape[0]*predictions_edit2.shape[1],predictions_edit2.shape[2])
print(predictions_edit2.shape)
print('shape of true and predict ',y_label2.shape,predictions.shape)

#perform model evaluation
eval_results = eval_metrics_on(predictions,y_label2)
print('this is evaluation results for r2,mean_abs_error,mse,rmse,expl_var_score ',eval_results)
#print('this is evaluation results for r2,mean_abs_error,mse,rmse,expl_var_score ',eval_results)

#observe the graph between prediction and true value of PM2.5 for last hour
y_true = y_label[:,2:3,:]
y_true = y_true.reshape(y_true.shape[0]*y_true.shape[1],y_true.shape[2])
pred_graph = predictions_edit[:,2:3,:]
pred_graph =pred_graph.reshape(pred_graph.shape[0]*pred_graph.shape[1],pred_graph.shape[2])
predict_graph(y_true,pred_graph)

print('rmse value')
stat_dict = {}
stat_avg = []
for i in range(time_step1):
    t1 = predictions_edit2[:,i:i+1,:]
    t1 = t1.reshape(t1.shape[0]*t1.shape[1],t1.shape[2])
    t2 = y_label[:,i:i+1,:]
    t2 = t2.reshape(t2.shape[0]*t2.shape[1],t2.shape[2])
    rsl = []
    for sn in range(number_stations):
        name = 'stats_' + str(sn)
        score2 = np.sqrt(np.mean(np.square(t1[:,sn::number_stations]-t2[:,sn::number_stations])))
        stat_dict.setdefault(name,[]).append(score2)
        #print(score2)
        rsl.append(score2)
for k in  stat_dict.keys():
    avg_value = sum(stat_dict[k])/time_step1
    #print(avg_value)
    stat_avg.append(avg_value)
print('mean of all stations ',mean(stat_avg))

#save the prediction reuslts
np.save('strip_4tilepredresults_next3day.npy',predictions_edit2)

In [ ]:
from tensorflow.keras import backend as K
from tensorflow import keras

import threading
import argparse
import numpy as np
import math
import time
import gc
from collections import Iterable
from tensorflow.keras import losses
from statistics import mean
from tensorflow.keras.utils import plot_model
import tensorflow.python.keras.backend as K

from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import TensorBoard,ModelCheckpoint, EarlyStopping,Callback
from tensorflow.keras.regularizers import L1L2
from tensorflow.keras.optimizers import *
#K.set_image_data_format('channels_first')

import seaborn as sns
sns.set()

import os


import tensorflow.compat.v1 as tf
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
gpu_options = tf.GPUOptions(allow_growth=True)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

start_time = time.time()


n_classes = 18
time_step = 3
#optim = Adam(lr = 0.0001)
#optim = Adam(lr = 0.00001, decay = 0.00001)
optim = Adam(learning_rate = 1e-4)
#sample_batch = 16
sample_batch = 10
number_stations = 18


class ClearMemory(Callback):
    def on_epoch_end(self, epoch, logs=None):
        gc.collect()
        K.clear_session()
        
        
        
aod_train = np.load('middlelayer_train_3day_4tile.npy')[0:350].astype('float')

local_weather_train14 = np.load('local_weather_yr14_n3day_train.npy').astype('float')

local_weather_train14 = local_weather_train14.reshape(local_weather_train14.shape[0],local_weather_train14.shape[1],
                                                  local_weather_train14.shape[2]*local_weather_train14.shape[3])

local_pm = np.load('pm_n3day_daily_train.npy')[0:353].astype('float')
tru_val = np.load('pm_label_n3day_daily_train.npy')[0:353].astype('float')

model2 = load_model('next3day_strip_model.h5')
predictions_edit2 = model2.predict([aod_train,local_pm[:-3],local_weather_train14[:-3]],
                            batch_size = sample_batch, verbose=1)


print(predictions_edit2.shape)

np.save('strip_4tilepredresults_2014_next3day.npy',predictions_edit2)

In [ ]:
from tensorflow.keras import backend as K
from tensorflow import keras

import threading
import argparse
import numpy as np
import math
import time
import gc
from collections import Iterable
from tensorflow.keras import losses
from statistics import mean
from tensorflow.keras.utils import plot_model
import tensorflow.python.keras.backend as K

from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import TensorBoard,ModelCheckpoint, EarlyStopping,Callback
from tensorflow.keras.regularizers import L1L2
from tensorflow.keras.optimizers import *
#K.set_image_data_format('channels_first')

import seaborn as sns
sns.set()

import os


import tensorflow.compat.v1 as tf
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
gpu_options = tf.GPUOptions(allow_growth=True)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

start_time = time.time()


n_classes = 18
time_step = 3
#optim = Adam(lr = 0.0001)
#optim = Adam(lr = 0.00001, decay = 0.00001)
optim = Adam(learning_rate = 1e-4)
#sample_batch = 16
sample_batch = 10
number_stations = 18


class ClearMemory(Callback):
    def on_epoch_end(self, epoch, logs=None):
        gc.collect()
        K.clear_session()
        
        
        
aod_train = np.load('middlelayer_train_3day_4tile.npy')[350:700].astype('float')

local_weather_train15 = np.load('local_weather_yr15_n3day_train.npy').astype('float')

local_weather_train14 = local_weather_train14.reshape(local_weather_train14.shape[0],local_weather_train14.shape[1],
                                                  local_weather_train14.shape[2]*local_weather_train14.shape[3])

local_pm = np.load('pm_n3day_daily_train.npy')[353].astype('float')
tru_val = np.load('pm_label_n3day_daily_train.npy')[0:353].astype('float')

model2 = load_model('next3day_strip_model.h5')
predictions_edit2 = model2.predict([aod_train,local_pm[:-3],local_weather_train14[:-3]],
                            batch_size = sample_batch, verbose=1)


print(predictions_edit2.shape)

np.save('strip_4tilepredresults_2015_next3day.npy',predictions_edit2)